In [60]:
import math
import time
import pandas as pd
import simplekml
import matplotlib.pyplot as plt
from sys import platform

#pd.set_option('mode.chained_assignment', None)

# Measuring execution time
start_time = time.time()

In [61]:
# Checks what operating system the user is using
if platform == 'win32':
    root_path = r'C:\Users\Stefanos\DeLasCasas CP, LLc\Ro De Las Casas - ' \
                r'DLC2P Server\AA Stefanos\Data Science\Python Projects\CIS ' \
                r'Data to Google Earth'
elif platform == 'darwin':
    root_path = r'/Users/Stefanos/Library/CloudStorage/' \
                r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/Ro De Las Casas ' \
                r'- DLC2P Server/AA Stefanos/Data Science/Python Projects/' \
                r'CIS Data to Google Earth'

In [62]:
# Create dataframe
df_cis = pd.read_csv(root_path + r'\Data\Data.csv')

# Specify Variables

In [63]:
# SPECIFY SCALE FACTOR
scale_factor = 20
# SPECIFY TRANSPARENCY (0 to 1)
transparency = 1
# SPECIFY ICON SIZE
icon_scale = 0.3
# SPECIFY COLOR PALETTE
# noinspection SpellCheckingInspection
my_cmap = plt.get_cmap('Spectral_r')
# SPECIFY LENGTH (ft)
distance = 5280
# SPECIFY MILES PER GRAPH
miles_per_graph = 1
# PROJECT NAME
project_name = 'XYZ'    #input('\nPROJECT NAME:')

# On

In [64]:
step = df_cis['STATIONING (ft)'].iloc[2]-df_cis['STATIONING (ft)'].iloc[1]
data_points = int(distance/step)

In [65]:
# Preventing chained assignment
df_cis_on = df_cis.copy()
#df_cis_on = df_cis[df_cis['PS_ON'] != 0]
#df_cis_on = df_cis_on.iloc[:, [False, True, False, False, True, True, False]]
df_cis_on = df_cis_on[['LONGITUDE', 'LATITUDE', 'PS_ON']].reset_index(drop=True)
df_cis_on['PS_ON'] = df_cis_on['PS_ON']*(-1)
total_miles = math.ceil(max(df_cis['STATIONING (ft)'])/distance)

In [66]:
# Counters
mile_number = 1
starting_point = 0
ending_point = starting_point + data_points
i = 0                                               # Row

while mile_number <= total_miles:
    kml = simplekml.Kml()
    style = simplekml.Style()
    z = df_cis_on.iloc[starting_point:ending_point].last_valid_index()

    while i < z:
        pnt = kml.newpoint(name=df_cis_on.loc[i, 'PS_ON']*(-1))
        pnt.style.balloonstyle.text = 'Potential: -{} V'.format(df_cis_on.loc[i, 'PS_ON'])
        pnt.coords = [(df_cis_on.loc[i, 'LONGITUDE'], df_cis_on.loc[i, 'LATITUDE'], df_cis_on.loc[i, 'PS_ON']*scale_factor)]
        # noinspection SpellCheckingInspection
        pnt.altitudemode = simplekml.AltitudeMode.relativetoground

        # Style
        pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/paddle/red-square-lv.png'
        pnt.style.iconstyle.scale = icon_scale
        pnt.style.labelstyle.scale = 0
        pnt.extrude = 1

        i += 1

    mile_number += miles_per_graph
    starting_point = ending_point
    ending_point += data_points
    i = 0

    kml.save(root_path + r'\Output/' + project_name + ' - ON (Mile ' + (mile_number-1).__str__() + ')' + '.kml')

# Off

In [34]:
kml = simplekml.Kml()

In [35]:
df_cis_off = df_cis[df_cis['PS_OFF'] != 0]
df_cis_off = df_cis_off.iloc[:, [False, False, True, False, True, True, False]]
df_cis_off = df_cis_off[['LONGITUDE', 'LATITUDE', 'PS_OFF']].reset_index(drop=True)
df_cis_off = df_cis_off.iloc[:2112, :]
df_cis_off['PS_OFF'] = df_cis_off['PS_OFF'] * -1

In [36]:
# Counters
i = 0                                   # Row
z = df_cis_off.last_valid_index()       # Max iterations

while i < z:
    pnt = kml.newpoint(name=df_cis_off.loc[i, 'PS_OFF']*-1)
    pnt.style.balloonstyle.text = 'Potential: -{} V'.format(df_cis_off.loc[i, 'PS_OFF'])
    pnt.coords = [(df_cis_off.loc[i, 'LONGITUDE'], df_cis_off.loc[i, 'LATITUDE'], df_cis_off.loc[i, 'PS_OFF']*scale_factor)]

    # noinspection SpellCheckingInspection
    pnt.altitudemode = simplekml.AltitudeMode.relativetoground

    # Style
    pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/paddle/red-blank.png'
    pnt.style.iconstyle.scale = icon_scale
    pnt.style.labelstyle.scale = 0

    i += 1

kml.save(root_path + r'\Output/' + 'OFF - ' + project_name + '.kml')

# Pipe

In [37]:
kml = simplekml.Kml()

In [38]:
df_cis_pipe = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
df_cis_pipe['Potential (V)'] = 0

In [39]:
# Counters
i = 0                                    # Row
z = 282                                  # Max iterations

while i < z:
    ls = kml.newlinestring()
    ls.style.balloonstyle.text = 'Ground'
    ls.coords = [(df_cis_pipe.loc[i, 'LONGITUDE'], df_cis_pipe.loc[i, 'LATITUDE'], df_cis_pipe.loc[i, 'Potential (V)']),
                 (df_cis_pipe.loc[i+1, 'LONGITUDE'], df_cis_pipe.loc[i+1, 'LATITUDE'], df_cis_pipe.loc[i+1, 'Potential (V)'])]

    # Style
    ls.style.linestyle.width = 5
    ls.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, 175)

    i += 1

kml.save(root_path + r'\Output/' + 'Ground - ' + project_name + '.kml')

# 1200

In [40]:
kml = simplekml.Kml()

In [41]:
df_cis_1200 = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
df_cis_1200['Potential (V)'] = 1.200

In [42]:
# Counters
i = 0                                    # Row
z = 282                                  # Max iterations

while i < z:
    ls = kml.newlinestring()
    ls.style.balloonstyle.text = 'Potential: -{} V'.format(df_cis_1200.loc[i, 'Potential (V)'])
    ls.coords = [(df_cis_1200.loc[i, 'LONGITUDE'], df_cis_1200.loc[i, 'LATITUDE'], df_cis_1200.loc[i, 'Potential (V)']*scale_factor),
                 (df_cis_1200.loc[i+1, 'LONGITUDE'], df_cis_1200.loc[i+1, 'LATITUDE'], df_cis_1200.loc[i+1, 'Potential (V)']*scale_factor)] 
    ls.altitudemode = simplekml.AltitudeMode.relativetoground

    # Style
    ls.style.linestyle.width = 5
    ls.style.linestyle.color = simplekml.Color.rgb(175, 175, 175, 255)

    i += 1

kml.save(root_path + r'\Output/' + '1.2 V - ' + project_name + '.kml')

# 850

In [43]:
kml = simplekml.Kml()

In [44]:
df_cis_850 = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
df_cis_850['Potential (V)'] = 0.850

In [45]:
i = 0                                    # Row
z = 282                                  # Max iterations

while i < z:
    ls = kml.newlinestring()
    ls.style.balloonstyle.text = 'Potential: {} V'.format(df_cis_850.loc[i, 'Potential (V)'])
    ls.coords = [(df_cis_850.loc[i, 'LONGITUDE'], df_cis_850.loc[i, 'LATITUDE'], df_cis_850.loc[i, 'Potential (V)']*scale_factor),
                 (df_cis_850.loc[i+1, 'LONGITUDE'], df_cis_850.loc[i+1, 'LATITUDE'], df_cis_850.loc[i+1, 'Potential (V)']*scale_factor)]
    ls.altitudemode = simplekml.AltitudeMode.relativetoground

    # Style
    ls.style.linestyle.width = 5
    ls.style.linestyle.color = simplekml.Color.rgb(175, 175, 175, 255)

    i += 1

kml.save(root_path + r'\Output/' + '0.85 V - ' + project_name + '.kml')

# Comments

In [46]:
kml = simplekml.Kml()

In [47]:
df_cis_comments = df_cis.iloc[:, [False, False, False, True, True, True, False]].copy()

In [48]:
# Counters
i = 0                                  # Row
z = df_cis_comments.last_valid_index()       # Max iterations

while i < z:
    # noinspection SpellCheckingInspection
    if pd.isna(df_cis_comments.loc[i, 'COMMENTS']):
        i += 1

    else:
        pnt = kml.newpoint(name=df_cis_comments.loc[i, 'COMMENTS'])
        pnt.coords = [(df_cis_comments.loc[i, 'LONGITUDE'], df_cis_comments.loc[i, 'LATITUDE'], 0)]

        pnt.altitudemode = simplekml.AltitudeMode.relativetoground

        # Style
        pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/paddle/ltblu-blank.png'
        pnt.style.iconstyle.scale = icon_scale*0.05
        #pnt.style.labelstyle.scale = 0

        i += 1

kml.save(root_path + r'\Output/' + 'Comments - ' + project_name + '.kml')